In [1]:
import requests
import json
import os
from pyspark.sql import SparkSession
from datetime import datetime, timedelta
from IPython.display import HTML


spark = SparkSession.builder.getOrCreate()


# ----------------------------
# Funções auxiliares
# ----------------------------


def converter_para_api(data_ddmmaaaa):

    dia, mes, ano = data_ddmmaaaa.split("/")

    return f"{dia}{mes}{ano}"




def gerar_datas(inicio, fim):

    datas = []

    d_inicio = datetime.strptime(inicio, "%d/%m/%Y")

    d_fim = datetime.strptime(fim, "%d/%m/%Y")



    d = d_inicio

    while d <= d_fim:

        datas.append(d.strftime("%d/%m/%Y"))

        d += timedelta(days=1)



    return datas




# ----------------------------
# Intervalo desejado
# ----------------------------



data_inicio = "01/01/2025"

data_fim = "05/01/2025"



lista_datas = gerar_datas(data_inicio, data_fim)

registros = []


# ----------------------------
# LOCAL DE SALVAMENTO NO LAKEHOUSE
# ----------------------------


parquet_root = "Files/Voos/"          # <-- AGORA AQUI

raw_root = "Files/Voos/raw_json/"     # opcional


os.makedirs(raw_root, exist_ok=True)


# ----------------------------
# Ingestão
# ----------------------------


for data in lista_datas:

    data_api = converter_para_api(data)

    url = f"https://sas.anac.gov.br/sas/siros_api/voos?dataReferencia={data_api}"


    print(f"\n📡 Consultando {data} → {url}")

    resp = requests.get(url)

    raw = resp.text


    # data=YYYY-MM-DD

    data_formatada = datetime.strptime(data, "%d/%m/%Y").strftime("%Y-%m-%d")


    # ----------------------------
    # Salvar JSON RAW (opcional)
    # ----------------------------

    pasta_raw = f"{raw_root}/data={data_formatada}/"

    os.makedirs(pasta_raw, exist_ok=True)


    with open(f"{pasta_raw}/voos_raw.json", "w", encoding="utf-8") as f:

        f.write(raw)


    print(f"💾 RAW salvo em: {pasta_raw}voos_raw.json")


    # ----------------------------
    # Normalizar JSON para DF
    # ----------------------------

    try:

        dado = resp.json()

        if isinstance(dado, str):

            dado = json.loads(dado)


        if isinstance(dado, list):

            for r in dado:

                r["data_consulta"] = data_formatada

            registros.extend(dado)

    except:

        print(f"❌ JSON inválido em {data}")


# ----------------------------
# Criar DataFrame
# ----------------------------



df = spark.createDataFrame(registros)

print(f"\n📊 Total de registros coletados: {df.count()}")


# ----------------------------
# SALVAR PARQUET (BRONZE)
# ----------------------------



(

    df.write

        .mode("overwrite")          # ou append

        .format("parquet")

        .partitionBy("data_consulta")    # ✓ partição por data

        .save(parquet_root)

)



print("✅ PARQUET salvo em:")

print(parquet_root)


# ----------------------------
# Display helper
# ---------------------------


def display(df, n=20):

    pdf = df.limit(n).toPandas()

    styled = (

        pdf.style

            .set_table_styles([

                {

                    "selector": "table",

                    "props": [

                        ("border", "2px solid #ccc"),

                        ("border-collapse", "collapse"),

                        ("font-family", "Arial"),

                        ("font-size", "13px")

                    ]

                },

                {

                    "selector": "th",

                    "props": [

                        ("background-color", "#f2f2f2"),

                        ("padding", "7px"),

                        ("border", "2px solid #ccc")

                    ]

                },

                {

                    "selector": "td",

                    "props": [

                        ("padding", "7px"),

                        ("border", "2px solid #ccc")

                    ]

                }

            ])

            .set_table_attributes('style="display:inline-block; overflow-x:auto; max-width:100%;"')

    )



    return HTML(styled.to_html())



display(df)

StatementMeta(, 9722d707-d4fb-4a52-926d-c6bbf82d118d, 3, Finished, Available, Finished)


📡 Consultando 01/01/2025 → https://sas.anac.gov.br/sas/siros_api/voos?dataReferencia=01012025
💾 RAW salvo em: Files/Voos/raw_json//data=2025-01-01/voos_raw.json

📡 Consultando 02/01/2025 → https://sas.anac.gov.br/sas/siros_api/voos?dataReferencia=02012025
💾 RAW salvo em: Files/Voos/raw_json//data=2025-01-02/voos_raw.json

📡 Consultando 03/01/2025 → https://sas.anac.gov.br/sas/siros_api/voos?dataReferencia=03012025
💾 RAW salvo em: Files/Voos/raw_json//data=2025-01-03/voos_raw.json

📡 Consultando 04/01/2025 → https://sas.anac.gov.br/sas/siros_api/voos?dataReferencia=04012025
💾 RAW salvo em: Files/Voos/raw_json//data=2025-01-04/voos_raw.json

📡 Consultando 05/01/2025 → https://sas.anac.gov.br/sas/siros_api/voos?dataReferencia=05012025
💾 RAW salvo em: Files/Voos/raw_json//data=2025-01-05/voos_raw.json

📊 Total de registros coletados: 14047
✅ PARQUET salvo em:
Files/Voos/


,data_consulta,ds_tipo_servico,dt_chegada_prevista_utc,dt_partida_prevista_utc,dt_referencia,nr_etapa,nr_voo,qt_assentos_previstos,sg_empresa_icao,sg_equipamento_icao,sg_icao_destino,sg_icao_origem,tx_codeshare
0,2025-01-01,REGULAR DE PASSAGEIROS DOMÉSTICA,01/01/2025 12:10,01/01/2025 11:00,01/01/2025,1,0004,15,CQB,E110,SWKO,SBEG,
1,2025-01-01,REGULAR DE PASSAGEIROS DOMÉSTICA,01/01/2025 13:50,01/01/2025 12:40,01/01/2025,1,0005,15,CQB,E110,SBEG,SWKO,
2,2025-01-01,REGULAR DE PASSAGEIROS INTERNACIONAL,02/01/2025 00:55,01/01/2025 17:15,01/01/2025,1,0009,168,TAP,A21N,SBSG,LPPT,"AZU/7102, UAE/4818"
3,2025-01-01,REGULAR DE PASSAGEIROS INTERNACIONAL,02/01/2025 00:50,01/01/2025 17:00,01/01/2025,1,0011,269,TAP,A332,SBRF,LPPT,"AZU/7104, CBJ/5451, UAE/4826"
4,2025-01-01,REGULAR DE PASSAGEIROS INTERNACIONAL,01/01/2025 09:50,01/01/2025 02:20,01/01/2025,1,0012,298,TAP,A339,LPPT,SBRF,"AZU/7105, CBJ/5452, UAE/4827"
5,2025-01-01,REGULAR DE PASSAGEIROS INTERNACIONAL,01/01/2025 10:35,01/01/2025 00:45,01/01/2025,1,0014,243,AMX,B788,SBGR,MMMX,
6,2025-01-01,REGULAR DE PASSAGEIROS INTERNACIONAL,01/01/2025 22:35,01/01/2025 13:15,01/01/2025,1,0015,243,AMX,B788,MMMX,SBGR,
7,2025-01-01,REGULAR DE PASSAGEIROS INTERNACIONAL,01/01/2025 20:45,01/01/2025 07:05,01/01/2025,1,0015,329,THY,A359,SBGR,LTFM,
8,2025-01-01,REGULAR DE PASSAGEIROS INTERNACIONAL,02/01/2025 01:25,01/01/2025 22:20,01/01/2025,2,0015,329,THY,A359,SAEZ,SBGR,
9,2025-01-01,REGULAR DE PASSAGEIROS INTERNACIONAL,01/01/2025 05:30,01/01/2025 02:55,01/01/2025,1,0016,329,THY,A359,SBGR,SAEZ,
